<a href="https://colab.research.google.com/github/TapasKumarDutta1/IEEE-CIS-Fraud/blob/master/magic_feature/finding_magic_feature_d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive
import pandas as pd
import numpy as np
import gc
from tqdm import tqdm
import random


In [ ]:
os.environ['KAGGLE_USERNAME'] = "tapaskd123" # username from the json file
os.environ['KAGGLE_KEY'] = "aba8dc1f085221111d925003fe5a88ed" # key from the json file
!kaggle competitions download -c ieee-fraud-detection

 79% 41.0M/52.2M [00:01<00:00, 16.2MB/s]
100% 52.2M/52.2M [00:01<00:00, 33.8MB/s]
  0% 0.00/3.21M [00:00<?, ?B/s]
100% 3.21M/3.21M [00:00<00:00, 105MB/s]
 91% 53.0M/58.3M [00:00<00:00, 76.6MB/s]
100% 58.3M/58.3M [00:00<00:00, 91.8MB/s]
  0% 0.00/3.26M [00:00<?, ?B/s]
100% 3.26M/3.26M [00:00<00:00, 107MB/s]
  0% 0.00/1.14M [00:00<?, ?B/s]
100% 1.14M/1.14M [00:00<00:00, 159MB/s]


In [ ]:
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
Mounted at /content/gdrive


In [ ]:
trn=pd.read_csv('train_transaction.csv.zip')
trn_idn=pd.read_csv('train_identity.csv.zip')

In [ ]:
trn=trn.merge(trn_idn,on='TransactionID',how='left')
del([trn_idn])
gc.collect()

11

In [ ]:
trn.head()

TransactionID  isFraud  ...  DeviceType                     DeviceInfo
0        2987000        0  ...         NaN                            NaN
1        2987001        0  ...         NaN                            NaN
2        2987002        0  ...         NaN                            NaN
3        2987003        0  ...         NaN                            NaN
4        2987004        0  ...      mobile  SAMSUNG SM-G892A Build/NRD90M

[5 rows x 434 columns]

In [ ]:
trn['day']=trn['TransactionDT']//86400
for col in range(1,16):
  trn['d_'+str(col)]=trn['day']-trn['D'+str(col)]

In [ ]:
lst=[]
for i in range(1,16):
  lst.append('d_'+str(i))
lst.append('TransactionAmt')
lst.append('V307')


In [ ]:
trn=trn.drop(['TransactionID','TransactionDT','isFraud'],1)
cat=list(trn.select_dtypes(include=object))
trn[cat]=trn[cat].fillna('nan')
trn=trn.fillna(-999)

In [ ]:
trn=trn.loc[trn['D1']<trn['day']].reset_index(drop=True)

In [ ]:
def group(df, cols):
    df["id"] = ""
    for col in cols:
        df["id"] += df[col].astype(str)
    result = 0
    for _, sdo in df.groupby(["id"]):
        tmp = 0
        tmp += sdo["TransactionAmt"].values.sum()
        tmp -= sdo.tail(1)["V307"].values
        if tmp > 0:
            result += tmp
        else:
            tmp -= result
    return result


trn = trn[lst]
cols = list(trn)
cols.remove("TransactionAmt")
cols.remove("V307")
random.shuffle(cols)
ans = 0
ls = []
dk = {}
for en, col in enumerate(tqdm(cols)):
    ls.append(col)
    n_ans = group(trn, ls)
    dk[col] = ans - n_ans
    if ans != 0:
        if n_ans >= ans:
            ls.remove(col)
        else:
            ans = n_ans
    else:
        ans = n_ans

100%|██████████| 15/15 [13:01<00:00, 52.10s/it]


In [ ]:
ans

array([44264531.43782868])

In [ ]:
df=pd.DataFrame(dk).T
df['rank']=list(range(len(cols)))
df.sort_values(by=0,ascending=False).head(20)

0  rank
d_3   3.146593e+06     9
d_2   2.985458e+06     7
d_4   2.933751e+06     6
d_10  1.748745e+06     8
d_8   1.277028e+06     1
d_11  1.010622e+06    12
d_1   4.802301e+05    10
d_5   2.671833e+05    14
d_14  2.099799e+05     2
d_12  6.272938e+04     3
d_6   5.024102e+04     4
d_7   8.036399e+03    11
d_13  7.146864e+03     5
d_9   5.350625e+03    13
d_15 -5.845763e+07     0

In [ ]:
ls

['d_15',
 'd_8',
 'd_14',
 'd_12',
 'd_6',
 'd_13',
 'd_4',
 'd_2',
 'd_10',
 'd_3',
 'd_1',
 'd_7',
 'd_11',
 'd_9',
 'd_5']